# General Settings

In [ ]:
SETTING_RETRAIN_CLASSIFIER = False         # Change this to retrain the classifier
SETTING_SAVE_RETRAINED_CLASSIFIER = True   # Change this to save the retrained classifier (if retraining)
SETTING_LOAD_TRAINED_CLASSIFIER = True     # Change this in case SETTING_SAVE_RETRAINED_CLASSIFIER and you want to use the freshly trained classifier without overwriting the one on disc
SETTING_TEST_CLASSIFIER_ON_TEST_IMAGES = False  # Change this to show how the classifier works on test images

# Imports

## Note: Uses OpenCV 3.2 with Contrib (pip install opencv-contrib-python)

In [2]:
# SKLearn
import sklearn
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC 
from sklearn import svm
from skimage.feature import hog
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix, classification_report


# MatPlotLib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# The rest
import numpy as np
from scipy.ndimage.measurements import label
import cv2
from glob import glob
import random
import csv
import time

/Users/rbaron/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Define Feature Extraction Pipeline

In [3]:
def convert_color(img, color_space='LUV'):
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: 
        feature_image = np.copy(img)
    return feature_image   

def get_hog_features(img, 
                     orient,
                     pix_per_cell, 
                     cell_per_block,
                     vis=False, 
                     feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=False, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
        return features

def bin_spatial(img, size=(32, 32)):
    color1 = cv2.resize(img[:,:,0], size, interpolation=cv2.INTER_LINEAR).ravel()
    color2 = cv2.resize(img[:,:,1], size, interpolation=cv2.INTER_LINEAR).ravel()
    color3 = cv2.resize(img[:,:,2], size, interpolation=cv2.INTER_LINEAR).ravel()
    return np.hstack((color1, color2, color3))
                        
def color_hist(img, nbins=32, bins_range=(0, 256)):    #
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    return hist_features

In [4]:
IMG_SHAPE = (64, 64)

color_space = 'LUV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 12  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 1 # HOG cells per block
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
spatial_size = (48, 48) # Spatial binning dimensions
hist_bins = 64    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off

In [5]:
def extract_features(img, color_space=color_space, spatial_size=spatial_size, hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                        spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat):   
    img_resize = cv2.resize(img, spatial_size, interpolation=cv2.INTER_LINEAR)
    img_resize = (np.sqrt(img_resize.astype(np.float32)/255)*255).astype(np.uint8)    
    file_features = []   
    feature_image = convert_color(img_resize, color_space=color_space)
    file_features.append(feature_image.ravel())
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        file_features.append(spatial_features)
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        file_features.append(hist_features)
    if hog_feat == True:
    # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block)
        # Append the new feature vector to the features list
    file_features.append(hog_features)   
    #  print(len(file_features), len(spatial_features), len(hist_features), len(hog_features)) 
    return np.concatenate(file_features)

# Load training data for the classifier

In [6]:
if SETTING_RETRAIN_CLASSIFIER == True:
    # total_imgs = 0
    SAMPLES_TO_USE = 8000

    vehicle_fnames = glob('dataset/vehicles/*/*.png')
    non_vehicle_fnames = glob('dataset/non-vehicles/*/*.png')

    vehicle_fnames = random.sample(vehicle_fnames, SAMPLES_TO_USE)
    non_vehicle_fnames = random.sample(non_vehicle_fnames, SAMPLES_TO_USE)

    print("Sampling complete, loading images")

    car_images = [mpimg.imread(fname) for fname in vehicle_fnames]
    non_car_images = [mpimg.imread(fname) for fname in non_vehicle_fnames]

    # Normalize
    car_images = [(image.astype(np.float32)/np.max(image)*255).astype(np.uint8) for image in car_images]
    non_car_images = [(image.astype(np.float32)/np.max(image)*255).astype(np.uint8) for image in non_car_images]

    print("Loading complete, extracting features")

    car_features = [extract_features(image) for image in car_images]
    non_car_features = [extract_features(image) for image in non_car_images]

    print("Extracting complete")

    X_cars = np.vstack(car_features)
    y_cars = np.ones(X_cars.shape[0], dtype=np.uint8)
    X_non_cars = np.vstack(non_car_features)
    y_non_cars = np.zeros(X_non_cars.shape[0], dtype=np.uint8)

    X = np.vstack((X_cars, X_non_cars))
    y = np.concatenate((y_cars, y_non_cars))


# Normalize training data

In [7]:
if SETTING_RETRAIN_CLASSIFIER == True:
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)

    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)

    print('{} cars labeled & {} non-cars labeled'.format(len(X_cars), len(X_non_cars)))
    del X_cars
    del X_non_cars

    # Split up data into randomized training and test sets
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)

# Train the classifier

## Search parameters

In [8]:
# After trying an (nearly) infite number of kernels via grid search... I  settle on the best performing one
SEARCH_C_VALUES=[0.0001]
SEARCH_KERNEL_VALUES=['linear'] 
SEARCH_GAMMA_VALUES=[0.00001]
parameters = {'kernel': SEARCH_KERNEL_VALUES, 'C': SEARCH_C_VALUES, 'gamma': SEARCH_GAMMA_VALUES}

## Grid search (commented out because we have a good parameter combination now)

In [9]:
# def optimize_param(parameters):
#     svc = GridSearchCV(svm.SVC(), parameters)
#     # Check the training time for the SVC
#     t=time.time()
#     svc.fit(X_train, y_train)
#     t2 = time.time()
#     print(round(t2-t, 2), 'Seconds to train SVC...')
#     optimal_params = svc.best_params_
#     print("Optimal parameters found: ", optimal_params, "\n")
#     # Check the score of the SVC
#     test_score = round(svc.score(X_test, y_test), 4)
#     print('Test Accuracy of SVC = ', test_score)
#     # Check the prediction time for a single sample
#     t=time.time()

## Final training

In [10]:
if SETTING_RETRAIN_CLASSIFIER == True:
    print('Using:',orient,'orientations',pix_per_cell,
        'pixels per cell and', cell_per_block,'cells per block')
    print('Feature vector length:', len(X_train[0]))

    svca = LinearSVC(C=0.0001, dual=False, max_iter=5)


    Batch_size = len(X_train)//5
    print('training on {} samples, {} batches, each batch {} samples'.format(len(X_train), len(X_train)//Batch_size, Batch_size))

    for batch in range(0,len(X_train)//Batch_size):
    # for batch in range(0,2):
        t=time.time()
        svca.fit(X_train[batch*Batch_size:(batch+1)*Batch_size], y_train[batch*Batch_size:(batch+1)*Batch_size])
        t2 = time.time()
        print('batch:', batch+1,'-', round(t2-t, 2), 'Seconds to train SVC...')

        # Check the score of the SVC
        test_score = round(svca.score(X_test, y_test), 4)
        print('Test Accuracy of SVC = ', test_score)
        # Check the prediction time for a single sample
        t=time.time()
    print ('done')

## Print statistics

In [11]:
if SETTING_RETRAIN_CLASSIFIER == True:
    predict = svca.predict(X_test)

    def print_stats(labels, predict):
        print(labels[:20])
        print(predict[:20])
        cm = confusion_matrix(labels, predict)
        tot = cm.sum()
        TN = cm[0][0]/tot
        FP = cm[0][1]/tot
        FN = cm[1][0]/tot
        TP = cm[1][1]/tot
        print("%s %.2f%% %s %.2f%% %s %.2f%% %s %.2f%%\n" % ('TP:',TP, 'FP:',FP, 'TN:',TN, 'FN:',FN))   
        print(classification_report(labels, predict))

    print_stats(y_test, predict)

# Save the classifier if we're happy with it

In [12]:
CLASSIFIER_SAVE_PATH = 'intermediates/car_classifier.pkl'
SCALER_SAVE_PATH = 'intermediates/car_scaler.pkl'

if SETTING_SAVE_RETRAINED_CLASSIFIER == True and SETTING_RETRAIN_CLASSIFIER == True:
    joblib.dump(svca, CLASSIFIER_SAVE_PATH)
    joblib.dump(X_scaler, SCALER_SAVE_PATH)

# Load the saved classifier

In [13]:
SETTING_LOAD_TRAINED_CLASSIFIER = True
CLASSIFIER_LOAD_PATH = 'intermediates/car_classifier.pkl'
SCALER_LOAD_PATH = 'intermediates/car_scaler.pkl'
if SETTING_LOAD_TRAINED_CLASSIFIER == True:
    svc = None
    svc = joblib.load(CLASSIFIER_LOAD_PATH)
    if svc != None:
        print('Classifier loaded successfully:')
        print(svc)
    X_scaler = joblib.load(SCALER_LOAD_PATH)
    if X_scaler != None:
        print('Scaler loaded successfully:')
        print(X_scaler)

Classifier loaded successfully:
LinearSVC(C=0.0001, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=5,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
Scaler loaded successfully:
StandardScaler(copy=True, with_mean=True, with_std=True)


# Define the window search and conversion to bounding boxes

In [14]:
WINDOW_SEARCH_RANGES = [
#      {'window_size': (256,256), 'x_start_stop': [None, None], 'y_start_stop': [360, 740], 'xy_overlap':(0.8, 0.8)},
     {'window_size': (192,192), 'x_start_stop': [None, None], 'y_start_stop': [360, 740], 'xy_overlap':(0.8, 0.8)},
     {'window_size': (128,128), 'x_start_stop': [50, 1300], 'y_start_stop': [360, 600], 'xy_overlap':(0.8, 0.8)},
     {'window_size': (64,64), 'x_start_stop': [200, 1200], 'y_start_stop': [360, 500], 'xy_overlap':(0.85, 0.85)},
]

def search_windows(img, windows, clf, scaler, color_space=color_space, spatial_size=spatial_size, hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                        spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat):

    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        #3) Extract the test window from original image
#         print(window)
        test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], IMG_SHAPE)
        
        #4) Extract features for that window using single_img_features()
        features = extract_features(test_img, color_space=color_space, spatial_size=spatial_size, hist_bins=hist_bins, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, hog_channel=hog_channel,
                        spatial_feat=spatial_feat, hist_feat=hist_feat, hog_feat=hog_feat)
        
        #5) Scale extracted features to be fed to classifier
        test_features = scaler.transform(np.array(features).reshape(1, -1))
        
        #6) Predict using your classifier
        prediction = clf.predict(test_features)
        #7) If positive (prediction == 1) then save the window
        if prediction == 1:
            on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows

def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=4):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

def heatmap_from_positives(image, positive_windows):
    heatmap = np.zeros(image.shape[0:2])
    for window in positive_windows:
        heatmap[window[0][1]:window[1][1], window[0][0]:window[1][0]] += 1
    return heatmap

def get_all_windows(image):
    all_windows = []
    for search_range in WINDOW_SEARCH_RANGES:
        windows = slide_window(image, 
                               xy_window=search_range['window_size'],
                               x_start_stop=search_range['x_start_stop'], 
                               y_start_stop=search_range['y_start_stop'], 
                               xy_overlap=search_range['xy_overlap'])
        all_windows.extend(windows)
    return all_windows

# def get_var_windows(window_sizes):
#     ret_windows = []
#     for size in window_sizes:
#         ret_windows.extend(slide_window(image, x_start_stop=[None, None], y_start_stop=y_start_stop, 
#                     xy_window=(size, size), xy_overlap=search_windows_overlap))
#     return ret_windows

def get_labeled_bboxes(labels):
    bboxes = []
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        bboxes.append(bbox)
    return bboxes

def get_bbox_center(bbox):
    x = (bbox[1][0] - bbox[0][0])/2 + bbox[0][0]
    y = (bbox[1][1] - bbox[0][1])/2 + bbox[0][1]
    return (int(x),int(y))

# test_bb = ((100, 500), (300, 700))
# test = get_bbox_center(test_bb)
# print(test)

# Try classifier on provided test images

In [15]:
DETECTION_THRESHOLD = 4

if SETTING_TEST_CLASSIFIER_ON_TEST_IMAGES:
    files = glob('test_images/test*.jpg')
    for file in files:
        image = mpimg.imread(file)
        draw_image = np.copy(image)

        windows = get_all_windows(image)
        
        hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                                spatial_size=spatial_size, hist_bins=hist_bins, 
                                orient=orient, pix_per_cell=pix_per_cell, 
                                cell_per_block=cell_per_block, 
                                hog_channel=hog_channel, spatial_feat=spatial_feat, 
                                hist_feat=hist_feat, hog_feat=hog_feat)                       


        all_windows_img = draw_boxes(draw_image, windows)
        positive_labeled_windows = draw_boxes(draw_image, hot_windows)

        heatmap = heatmap_from_positives(image, hot_windows)
        heatmap_orig = np.copy(heatmap)
        # Threshold
        heatmap[heatmap <= DETECTION_THRESHOLD] = 0
        # Label separate detections
        labels = label(heatmap)
        # Calculate bounding boxes for labels
        detected_bboxes = get_labeled_bboxes(labels)
        # Draw bounding boxes on image
        if detected_bboxes is not None:
            img_with_bboxes = draw_boxes(image, detected_bboxes)
        else: 
            img_with_bboxes = image

        # Plot
        f, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(24, 10))
        f.tight_layout()
        ax1.set_title('Scanned Windows', fontsize=30)
        ax1.imshow(all_windows_img)
        ax2.set_title('Positive Scans', fontsize=30)
        ax2.imshow(positive_labeled_windows)
        ax3.set_title('Heatmap before threshold', fontsize=30)
        ax3.imshow(heatmap_orig, cmap='gray')
        ax4.set_title('Heatmap after threshold', fontsize=30)
        ax4.imshow(heatmap, cmap='hot')
        ax5.set_title('Labels', fontsize=30)
        ax5.imshow(labels[0], cmap='gray')
        ax6.set_title('Final Detections', fontsize=30)
        ax6.imshow(img_with_bboxes)
        plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.1, hspace=0.2)
        plt.show()

# Define and test some bounding box manipulation functions

In [ ]:
# Shift a bounding box by a horizontal and vertical shift but retain its size
# Drops the window (returns None) if any of its coordinates cross the keep_bbox
def shift_bbox(bbox, horizontal_shift, vertical_shift, keep_bbox):
    p1 = bbox[0]
    p1_x = p1[0]
    p1_y = p1[1]
    p2 = bbox[1]
    p2_x = p2[0]
    p2_y = p2[1] 
    
    new_p1_x = p1_x+horizontal_shift
    new_p1_y = p1_y+vertical_shift
    new_p2_x = p2_x+horizontal_shift
    new_p2_y = p2_y+vertical_shift
    shifted = ((new_p1_x, new_p1_y),(new_p2_x, new_p2_y)) 
    if new_p1_x < keep_bbox[0][0] or new_p1_y < keep_bbox[0][1] or new_p2_x > keep_bbox[1][0] or new_p2_y > keep_bbox[1][1]:
        return None
    return shifted

# Create nearby boxes around a bbox. Used for incremental search
def bbox_to_nearby(bbox, steps_per_direction, step_overlap_horiz, step_overlap_vert, scale, keep_bbox):
    p1 = bbox[0]
    p1_x = p1[0]
    p1_y = p1[1]
    p2 = bbox[1]
    p2_x = p2[0]
    p2_y = p2[1]
    width = p2_x-p1_x
    height = p2_y-p1_y
    horizontal_step_size = int(width * (1-step_overlap_horiz))
    vertical_step_size = int(height * (1-step_overlap_vert))
    print("Step size for horiz", horizontal_step_size)
    print("Step size for vert", vertical_step_size)
    
    windows = []
    windows.append(bbox)
    # Create windows per step in each direction
    for step in range(1, steps_per_direction+1):
        horiz_shift = horizontal_step_size*step
        vert_shift = vertical_step_size*step
        
        left_win = shift_bbox(bbox, -horiz_shift, 0, keep_bbox)
        right_win = shift_bbox(bbox, +horiz_shift, 0, keep_bbox)
        top_win = shift_bbox(bbox, 0, -vert_shift, keep_bbox)     
        bottom_win = shift_bbox(bbox, 0, +vert_shift, keep_bbox)
        windows.extend([left_win, right_win, top_win, bottom_win])

        top_left_win = shift_bbox(bbox, -horiz_shift, -vert_shift, keep_bbox)
        top_right_win = shift_bbox(bbox, +horiz_shift, -vert_shift, keep_bbox)
        bottom_left_win = shift_bbox(bbox, -horiz_shift, +vert_shift, keep_bbox)     
        bottom_right_win = shift_bbox(bbox, +horiz_shift, +vert_shift, keep_bbox) 
        windows.extend([top_left_win, top_right_win, bottom_left_win, bottom_right_win])

    # Filter out None windows (windows dropped due to boundary)
    windows = [window for window in windows if window is not None]
    return windows

keep_bbox = ((0,0),(1280, 720))
bbox1 = ((100,500),(200,700))
res = shift_bbox(bbox1, -10, 20, keep_bbox)
print(res) # should print ((90, 520), (190, 720))
res = shift_bbox(bbox1, 10, 30, keep_bbox)
print(res) # should print "None"
ress = bbox_to_nearby(bbox1, 1, 0.9, 0.9, 1, keep_bbox)
print(ress)

((90, 520), (190, 720))
None
Step size for horiz 9
Step size for vert 19
[((100, 500), (200, 700)), ((91, 500), (191, 700)), ((109, 500), (209, 700)), ((100, 481), (200, 681)), ((100, 519), (200, 719)), ((91, 481), (191, 681)), ((109, 481), (209, 681)), ((91, 519), (191, 719)), ((109, 519), (209, 719))]


# Use the classifier on the video

In [ ]:
def detect_cars_in_frame(image, windows):
    hot_windows = search_windows(image, 
                                 windows, 
                                 svc, 
                                 X_scaler, 
                                 color_space=color_space, 
                                 spatial_size=spatial_size, 
                                 hist_bins=hist_bins, 
                                 orient=orient, 
                                 pix_per_cell=pix_per_cell, 
                                 cell_per_block=cell_per_block, 
                                 hog_channel=hog_channel, 
                                 spatial_feat=spatial_feat, 
                                 hist_feat=hist_feat,
                                 hog_feat=hog_feat)                       

    heatmap = heatmap_from_positives(image, hot_windows)
    # Threshold
    heatmap[heatmap <= DETECTION_THRESHOLD] = 0
    # Label separate detections
    labels = label(heatmap)
    # Calculate bounding boxes for labels
    detected_bboxes = get_labeled_bboxes(labels)
    
    return detected_bboxes


        

video = cv2.VideoCapture('test_video.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_out = cv2.VideoWriter('test_video_processed.avi', fourcc, fps=20.0, frameSize=(1280,720), isColor=True)

FRAMES_BATCH_SIZE = 20  
NEARBY_WINDOW_OVERLAP_HORIZ = 0.95
NEARBY_WINDOW_OVERLAP_VERT = 0.98
NEARBY_WINDOW_STEPS_PER_DIRECTION = 2
NEARBY_WINDOW_SCALEDOWN = 0.98

has_reached_end = False
previous_batch_detections = None
full_scan_windows = None
frame_idx = 0
trackers = []
while(video.isOpened() and not has_reached_end):
    time_op_start = time.time()

    # Read a frame, make sure we're not at the end
    ret, frame = video.read()
    if ret == False:
        has_reached_end = True
        break
       
    # Get the full scan windows for the first time if needed
    if full_scan_windows is None:
        full_scan_windows = get_all_windows(frame)
    
    # For FRAMES_BATCH_SIZE frames, do full detection. 
    # Otherwise, just continue detecting using the trackers
    frame_detections = []
    if frame_idx % FRAMES_BATCH_SIZE == 0:
        print("Batch frame found")
        frame_detections = detect_cars_in_frame(
            frame,
            full_scan_windows
        )
        print("Creating trackers for detections")

        for detection in frame_detections:
            print("Creating Tracker")
            tracker = cv2.Tracker_create("KCF")
            tracker_bbox = (
                detection[0][0], 
                detection[0][1], 
                detection[1][0] - detection[0][0],
                detection[1][1] - detection[0][1]
            )
            print("Initializing tracker with bbox: ", tracker_bbox)
            tracker.init(frame, tracker_bbox)
            trackers.append(tracker)
    else:
        print("Incremental frame found")

        for idx, tracker in trackers:
            ok, bbox = tracker.update(frame)
            print("Tracker {} new bbox: {}".format(ok, bbox))
            if ok:
                p1 = (int(bbox[0]), int(bbox[1]))
                p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
                frame_detections.append((p1[0], p1[1], p2[0], p2[1]))
            else:
                print("Tracker lost")
    

    out_frame = draw_boxes(frame, frame_detections)
    video_out.write(out_frame)
    
#     # Get a batch of frames of size FRAMES_BATCH_SIZE
#     frames_batch = []
#     for i in range(0, FRAMES_BATCH_SIZE):    
#         ret, frame = video.read()
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             has_reached_end = True
#             break
#         if ret==True:
#             frames_batch.append(frame)
#         else:
#             has_reached_end = True
#             break
        
#     if len(frames_batch) == 0:
#         break
        
#     if full_scan_windows is None:
#         full_scan_windows = get_all_windows(frames_batch[0])
        
#     detections_per_frame = {}
#     first_frame_detections = detect_cars_in_frame(
#         frames_batch[0],
#         full_scan_windows
#     )
#     detections_per_frame[0] = first_frame_detections
#     for i in range(1,len(frames_batch)):
#         prev_frame_detections = detections_per_frame[i-1]
#         frame_search_windows = []
#         for detection in prev_frame_detections:
#             nearby_windows = bbox_to_nearby(detection, 
#                                             NEARBY_WINDOW_STEPS_PER_DIRECTION, 
#                                             NEARBY_WINDOW_OVERLAP_HORIZ,
#                                             NEARBY_WINDOW_OVERLAP_VERT,
#                                             NEARBY_WINDOW_SCALEDOWN, 
#                                             ((0,0), (1280, 720))
#                                            )
#             frame_search_windows.extend(nearby_windows)
            
#         frame_detections = detect_cars_in_frame(
#             frames_batch[i], 
#             frame_search_windows
#         )   
#         detections_per_frame[i] = frame_detections
#         print(frame_detections)
#         frames_batch[i] = draw_boxes(frames_batch[i], frame_detections)
    
#     # Write resulting frames
#     for out_frame in frames_batch:
#         video_out.write(out_frame)
    
    # Original pipeline time per frame was 9 seconds on my laptop.
    # To build a real-time pipeline we need 2 orders of magnitute improvement (<16ms for 60fps)
    time_op_end = time.time()
    print('Time per frame: {}', (time_op_end - time_op_start))

#     print('Time per frame: {}', (time_op_end - time_op_start)/len(frames_batch))
  

print("Releasing everything")
video.release()
video_out.release()
# cv2.destroyAllWindows()

Batch frame found


In [ ]:
# # import IPython.display as disp
# import base64
# # disp.Video()
# def video(fname, mimetype):
#     from IPython.display import HTML
#     video_not_encoded = open(fname, "rb").read()
#     video_encoded = base64.b64encode(video_not_encoded)
#     video_tag = '<video controls alt="test" src="data:video/{0};base64,{1}">'.format(mimetype, video_encoded.decode())
#     return HTML(data=video_tag)

# video("test_video_processed.avi", "avi")

# Trackers to try: KCF, MEDIANFLOW, GOTURN?